## Used Libraries

In [ ]:
#Real Code
import numpy as np
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm_notebook 

from sklearn.preprocessing import OneHotEncoder

## Setting random seed

In [ ]:
np.random.seed(0)

## Splitting and Normalisation

In [ ]:
(x_train,y_train),(x_val,y_val)=fashion_mnist.load_data()
x_train = x_train.astype('float')/ 255.0
x_val = x_val.astype('float')/ 255.0
print(x_train.shape, x_val.shape,y_train.shape,y_val.shape,type(x_train[0][0]),x_train[0][0])
x_train = x_train.reshape(60000,784)
x_val = x_val.reshape(10000,784)
print(x_train.shape, x_val.shape,y_train.shape,y_val.shape,type(x_train[0][0]),x_train[0][0])

(60000, 28, 28) (10000, 28, 28) (60000,) (10000,) <class 'numpy.ndarray'> [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
(60000, 784) (10000, 784) (60000,) (10000,) <class 'numpy.float64'> 0.0


## 10% for testing from training data (Terminology: I have kept aside validation data as hidden data and considered test data for testing accuracy)



In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1,stratify=y_train, random_state=0)
print(x_train.shape, x_test.shape,y_train.shape,y_test.shape,type(x_train[0][0]),x_train[0][0])
print(type(x_train))

(54000, 784) (6000, 784) (54000,) (6000,) <class 'numpy.float64'> 0.0
<class 'numpy.ndarray'>


## One Hot Encoding

In [ ]:
enc = OneHotEncoder()
# 0 -> (1, 0, 0, 0), 1 -> (0, 1, 0, 0), 2 -> (0, 0, 1, 0), 3 -> (0, 0, 0, 1)
y_OH_train = enc.fit_transform(np.expand_dims(y_train,1)).toarray()
y_OH_val = enc.fit_transform(np.expand_dims(y_test,1)).toarray()
print(y_OH_train.shape, y_OH_val.shape)

(54000, 10) (6000, 10)


## Main class (Fully Vectorized)

In [ ]:
class FFSN_MultiClass:
  
  def __init__(self, n_inputs, n_outputs, hidden_sizes=[3]):
    self.nx = n_inputs
    self.ny = n_outputs
    self.nh = len(hidden_sizes)
    self.sizes = [self.nx] + hidden_sizes + [self.ny] 

    self.W = {}
    self.B = {}
    for i in range(self.nh+1):
      self.W[i+1] = np.random.randn(self.sizes[i], self.sizes[i+1])
      self.B[i+1] = np.zeros((1, self.sizes[i+1]))
      
  def sigmoid(self, x):
    return 1.0/(1.0 + np.exp(-x))
  
  def softmax(self, x):
    exps = np.exp(x)
    return exps / np.sum(exps, axis=1).reshape(-1,1) 

  def forward_pass(self, x):
    self.A = {}
    self.H = {}
    self.H[0] = x
    for i in range(self.nh):
      self.A[i+1] = np.matmul(self.H[i], self.W[i+1]) + self.B[i+1]
      self.H[i+1] = self.sigmoid(self.A[i+1])
    self.A[self.nh+1] = np.matmul(self.H[self.nh], self.W[self.nh+1]) + self.B[self.nh+1]
    self.H[self.nh+1] = self.softmax(self.A[self.nh+1])
    return self.H[self.nh+1]
  
  def predict(self, X):
    Y_pred = []
    for x in X:
      y_pred = self.forward_pass(x)
      Y_pred.append(y_pred)
    return np.array(Y_pred).squeeze()
 
  def grad_sigmoid(self, x):
    return x*(1-x) 
  
  def cross_entropy(self,label,pred):
    yl=np.multiply(pred,label)
    yl=yl[yl!=0]
    yl=-np.log(yl)
    yl=np.mean(yl)
    return yl
 
  def grad(self, x, y):
    self.forward_pass(x)
    self.dW = {}
    self.dB = {}
    self.dH = {}
    self.dA = {}
    L = self.nh + 1
    self.dA[L] = (self.H[L] - y)
    for k in range(L, 0, -1):
      self.dW[k] = np.matmul(self.H[k-1].T, self.dA[k])
      self.dB[k] = np.sum(self.dA[k],axis=0).reshape(1,-1) #self.dA[k]
      self.dH[k-1] = np.matmul(self.dA[k], self.W[k].T)
      self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_sigmoid(self.H[k-1]))  
    
  def fit(self, X, Y, algo= "GD", epochs=100, initialize='True', learning_rate=0.01, display_loss=False,
          mini_batch_size=60000, eps=1e-8, beta=0.9, beta1=0.9, beta2=0.9, gamma=0.9):
      
    if display_loss:
      loss = {}
    V_W = {}
    V_B = {}
    M_W = {}
    M_B = {}
    num_updates = 0
    if initialize:
      for i in range(self.nh+1):
        V_W[i+1] = np.zeros((self.sizes[i], self.sizes[i+1]))
        V_B[i+1] = np.zeros((1, self.sizes[i+1]))
        M_W[i+1] = np.zeros((self.sizes[i], self.sizes[i+1]))
        M_B[i+1] = np.zeros((1, self.sizes[i+1]))
    if algo == 'SGD':
        for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
          points_seen = 0
          m = 1
          mini_batch_size = 1
          while(points_seen<X.shape[0]):
            self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
            m = mini_batch_size
            for i in range(self.nh+1):
              self.W[i+1] -= learning_rate * (self.dW[i+1]/m)
              self.B[i+1] -= learning_rate * (self.dB[i+1]/m)
            points_seen=points_seen+mini_batch_size           
          if display_loss:
            Y_pred = self.predict(X) 
            loss[epoch] = self.cross_entropy(Y, Y_pred)   
    
    if algo == 'Momentum':
      for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
        points_seen = 0
        m = mini_batch_size
        while(points_seen<X.shape[0]):
          self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
          for i in range(self.nh+1):
            V_W[i+1] = gamma * V_W[i+1] + learning_rate * (self.dW[i+1]/m)
            V_B[i+1] = gamma * V_B[i+1] + learning_rate * (self.dB[i+1]/m)
                    
          for i in range(self.nh+1):
            self.W[i+1] -= V_W[i+1]
            self.B[i+1] -= V_B[i+1]
          points_seen=points_seen+mini_batch_size
            
        if display_loss:
          Y_pred = self.predict(X) 
          loss[epoch] = self.cross_entropy(Y, Y_pred)

    if algo == 'NAG':
      for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
        points_seen = 0
        m = mini_batch_size
        for i in range(self.nh+1):
          self.W[i+1] -= gamma*V_W[i+1]
          self.B[i+1] -= gamma*V_B[i+1]
        while(points_seen<X.shape[0]):
          self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
          for i in range(self.nh+1):
            self.W[i+1] -= learning_rate * (self.dW[i+1]/m)
            self.B[i+1] -= learning_rate * (self.dB[i+1]/m)
          for i in range(self.nh+1):
            V_W[i+1] = gamma*V_W[i+1] + learning_rate * (self.dW[i+1]/m)
            V_B[i+1] = gamma*V_B[i+1] + learning_rate * (self.dB[i+1]/m)
            self.W[i+1] -= gamma*V_W[i+1]
            self.B[i+1] -= gamma*V_B[i+1]
          points_seen=points_seen+mini_batch_size 
        if display_loss:
          Y_pred = self.predict(X) 
          loss[epoch] = self.cross_entropy(Y, Y_pred)  

    if algo == 'RMSProp':
      for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
        points_seen = 0
        m = mini_batch_size
        while(points_seen<X.shape[0]):
          self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
          for i in range(self.nh+1):
            V_W[i+1] = beta*V_W[i+1] + (1-beta) * ((self.dW[i+1]/m)**2)
            V_B[i+1] = beta*V_B[i+1] + (1-beta) * ((self.dB[i+1]/m)**2)
            self.W[i+1] -= (learning_rate / np.sqrt(V_W[i+1]+ eps))* (self.dW[i+1]/m)
            self.B[i+1] -= (learning_rate / np.sqrt(V_B[i+1]+ eps))* (self.dB[i+1]/m)
          points_seen=points_seen+mini_batch_size
        if display_loss:
          Y_pred = self.predict(X) 
          loss[epoch] = self.cross_entropy(Y, Y_pred)
    if algo == 'Adam':
      for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
        points_seen = 0
        m = mini_batch_size
        while(points_seen<X.shape[0]):
          self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
          for i in range(self.nh+1):
            num_updates += 1
            M_W[i+1] = beta1*M_W[i+1] + (1-beta1) * (self.dW[i+1]/m)
            M_B[i+1] = beta1*M_B[i+1] + (1-beta1) * (self.dB[i+1]/m)
            V_W[i+1] = beta2*V_W[i+1] + (1-beta2) * ((self.dW[i+1]/m)**2)
            V_B[i+1] = beta2*V_B[i+1] + (1-beta2) * ((self.dB[i+1]/m)**2)
            M_W_C = M_W[i+1] / (1 - np.power(beta1, num_updates))
            M_B_C = M_B[i+1] / (1 - np.power(beta1, num_updates))
            V_W_C = V_W[i+1] / (1 - np.power(beta2, num_updates))
            V_B_C = V_B[i+1] / (1 - np.power(beta2, num_updates))
            self.W[i+1] -= (learning_rate / np.sqrt(V_W_C + eps))* M_W_C
            self.B[i+1] -= (learning_rate / np.sqrt(V_B_C + eps)) * M_B_C
          points_seen=points_seen+mini_batch_size
        
        if display_loss:
          Y_pred = self.predict(X)
          loss[epoch] = self.cross_entropy(Y, Y_pred)

    if algo == 'Nadam':
      for epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
        points_seen = 0
        m = mini_batch_size
        while(points_seen<X.shape[0]):
          num_updates += 1
          self.grad(X[points_seen:points_seen+mini_batch_size,:], Y[points_seen:points_seen+mini_batch_size,:])
          for i in range(self.nh+1):
            M_W[i+1] = beta1*M_W[i+1] + (1-beta1) * (self.dW[i+1]/m)
            M_B[i+1] = beta1*M_B[i+1] + (1-beta1) * (self.dB[i+1]/m)
            V_W[i+1] = beta2*V_W[i+1] + (1-beta2) * ((self.dW[i+1]/m)**2)
            V_B[i+1] = beta2*V_B[i+1] + (1-beta2) * ((self.dB[i+1]/m)**2)
            M_W_C = M_W[i+1] / (1 - np.power(beta1, num_updates))
            M_B_C = M_B[i+1] / (1 - np.power(beta1, num_updates))
            V_W_C = V_W[i+1] / (1 - np.power(beta2, num_updates))
            V_B_C = V_B[i+1] / (1 - np.power(beta2, num_updates))
            self.W[i+1] -= (learning_rate / np.sqrt(V_W_C + eps))* (beta1*M_W_C + (1-beta1)/(1-np.power(beta1,num_updates))* (self.dW[i+1]/m))
            self.B[i+1] -= (learning_rate / np.sqrt(V_B_C + eps)) * (beta1*M_B_C + (1-beta1)/(1-np.power(beta1,num_updates))* (self.dB[i+1]/m))
          points_seen=points_seen+mini_batch_size
        
        if display_loss:
          Y_pred = self.predict(X) 
          loss[epoch] = self.cross_entropy(Y, Y_pred)
    
    if display_loss:
        plt.plot(np.array(list(loss.values())).astype(float))
        plt.xlabel('Epochs')
        plt.ylabel('CE')
        plt.show()

## Creating Instance of the class and calling fit function

In [ ]:
ffsn_multi = FFSN_MultiClass(784,10,[128,128,128])
ffsn_multi.fit(x_train,y_OH_train,epochs=10,learning_rate=.0005,display_loss=True,algo= "Nadam",mini_batch_size=128)
#we can add new optimizer easily and it works with different batch sizes

## Accuracy

In [ ]:
Y_pred_train = ffsn_multi.predict(x_train)
Y_pred_train = np.argmax(Y_pred_train,1)

Y_pred_val = ffsn_multi.predict(x_test)
Y_pred_val = np.argmax(Y_pred_val,1)


accuracy_train = accuracy_score(Y_pred_train, y_train)
accuracy_val = accuracy_score(Y_pred_val, y_test)

print("Training accuracy", round(accuracy_train, 2))
print("Validation accuracy", round(accuracy_val, 2))

Training accuracy 0.86
Validation accuracy 0.84
